# Pitcher Data Demo

Inspired by: Thomas Nestico [(@TJStats)](https://x.com/TJStats)

The end result of this code will output a specific pitcher's outing from a specific game in Spring Training with specific values attributed to each type of pitch from the pitcher's pitch mix. This notebook will explain each attribute itself and how they are calculated. I wanted to try and replicate the incredible work that people like TJ put out for baseball fans so that I could better understand pitchers and the game we all love.

The end result will look as such:


In [5]:
import pandas as pd
pd.set_option("display.max_columns", None)  # Ensure all columns are displayed

df = pd.read_csv("pitch_type_counts.csv")

df

,Pitcher,Pitch Type,Count,Usage,Spin Rate,Avg Velo,iVB,HB,Whiffs,CS,CS+Whiffs,Zone%,Chase%,Whiff%,vRel,hRel,VAA,HAA,Extension,Max Exit Velo,Batter
0,Jon Gray,Four-Seam Fastball,20,50.0,1938.4,94.4,15.1,8.5,0,4,4,60.0,37.5,0.0,5.4,-1.6,-4.7,1.0,6.3,98.2,Lourdes Gurriel Jr.
1,Jon Gray,Slider,15,37.5,2473.7,86.7,1.9,-3.4,3,3,6,66.7,40.0,33.3,5.6,-1.6,-7.3,2.4,6.1,109.4,Alek Thomas
2,Jon Gray,Changeup,3,7.5,1502.7,87.3,8.1,14.2,1,0,1,66.7,0.0,50.0,5.5,-1.5,-6.7,0.6,6.3,NaN,NaN
3,Jon Gray,Curveball,2,5.0,2622.5,77.3,-8.3,-10.6,0,0,0,0.0,0.0,NaN,5.6,-1.5,-10.7,3.7,6.2,NaN,NaN


As we can see above, there are a lot of different attributes describing the pitches for Jon Gray.

We can see each pitch type that Jon has, as well as how many times he threw each pitch respectively (20 fastballs, 15 sliders, 3 changeups, 2 curveballs).

From that information we can calculate the next column, usage rate, by combining each pitch type count to give us a total amount of pitches thrown. With a total amount of pitches thrown, we can divide each pitch type count by the total to give us the usage rate.

I will go through this final output column by column and showcase the code used as well as an explanation for the code. 

In [6]:
# MLB Scraper Pitcher Data
import pandas as pd
import pybaseball as pyb
import numpy as np
from api_scraper import MLB_Scrape

# Set display options to print all columns without truncation
pd.set_option("display.max_columns", None)  # Ensure all columns are displayed
pd.set_option("display.max_rows", None)  # Display all rows, be cautious with large DataFrames
pd.set_option("display.width", None)  # Remove column width limit

y0 = 50  # Release y-position (feet)
yf = 17 / 12  # Home plate y-position (feet)

In [7]:
# Initialize the scraper
scraper = MLB_Scrape()

# Retrieve game data for the specific game ID
game_data = scraper.get_data(game_list_input=[778935])

# Convert the game data to a Polars DataFrame
data_df = scraper.get_data_df(data_list=game_data)

# Convert the Polars DataFrame to a Pandas DataFrame
pandas_df = data_df.to_pandas()

This May Take a While. Progress Bar shows Completion of Data Retrieval.


Processing: 100%|██████████| 1/1 [00:00<00:00,  1.72iteration/s]

Converting Data to Dataframe.


In [19]:
df_pyb = data_df.to_pandas()
df_pyb["PitchesThrown"] = 1
df_pyb.head(5)

,game_id,game_date,batter_id,batter_name,batter_hand,batter_team,batter_team_id,pitcher_id,pitcher_name,pitcher_hand,pitcher_team,pitcher_team_id,ab_number,play_description,play_code,in_play,is_strike,is_swing,is_whiff,is_out,is_ball,is_review,pitch_type,pitch_description,strikes,balls,outs,strikes_after,balls_after,outs_after,start_speed,end_speed,sz_top,sz_bot,x,y,ax,ay,az,pfxx,pfxz,px,pz,vx0,vy0,vz0,x0,y0,z0,zone,type_confidence,plate_time,extension,spin_rate,spin_direction,vb,ivb,hb,launch_speed,launch_angle,launch_distance,launch_location,trajectory,hardness,hit_x,hit_y,index_play,play_id,start_time,end_time,is_pitch,type_type,type_ab,event,event_type,rbi,away_score,home_score,PitchesThrown
0,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Ball,B,False,False,None,None,None,False,False,FF,Four-Seam Fastball,0,0,0,0,1,0,94.1,86.6,3.117,1.573,152.25,166.36,-7.708112,28.376567,-15.040927,-4.023972,8.943871,-0.924854,2.682155,4.094442,-137.064647,-4.440241,-1.910395,50.002223,5.339861,11,0.90,0.398585,6.271919,1919,223,-15.1,15.6,6.5,NaN,NaN,NaN,None,None,None,NaN,NaN,3,ac357149-abaf-4e84-a61d-2d357c6f3cbb,2025-03-02T20:07:20.629Z,2025-03-02T20:07:24.856Z,True,pitch,None,None,None,NaN,NaN,NaN,1
1,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Foul,F,False,True,True,None,None,False,False,FF,Four-Seam Fastball,0,1,0,1,1,0,93.9,85.9,3.117,1.573,105.55,187.36,-7.708316,30.294774,-15.543560,-4.084714,8.812437,0.300314,1.904322,6.615272,-136.558642,-6.412778,-1.623706,50.001922,5.353759,9,0.89,0.401204,6.274797,1846,223,-16.3,14.8,6.0,NaN,NaN,NaN,None,None,None,NaN,NaN,4,ac71bd8c-9247-4005-99f7-d5cd76fa41b4,2025-03-02T20:07:33.617Z,2025-03-02T20:07:36.863Z,True,pitch,None,None,None,NaN,NaN,NaN,1
2,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Ball,B,False,False,None,None,None,False,False,CH,Changeup,1,1,0,1,2,0,87.2,80.0,3.117,1.573,87.60,217.06,-13.549664,26.297841,-23.706664,-8.354960,5.221508,0.771382,0.804411,8.219174,-126.646911,-6.721460,-1.433305,50.004169,5.393922,14,0.93,0.432790,6.248571,1589,231,-28.1,8.0,13.2,NaN,NaN,NaN,None,None,None,NaN,NaN,5,0a82af13-2c2f-4115-8c65-59d719c6df8a,2025-03-02T20:07:48.825Z,2025-03-02T20:07:53.717Z,True,pitch,None,None,None,NaN,NaN,NaN,1
3,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Called Strike,C,False,True,None,None,None,False,True,FF,Four-Seam Fastball,1,2,0,2,2,0,94.4,86.6,3.117,1.573,146.74,180.56,-11.289134,29.538970,-16.268626,-5.883129,8.289911,-0.780242,2.156177,4.872686,-137.420980,-5.538319,-1.809204,50.004103,5.297263,4,0.90,0.398087,6.171662,1931,227,-16.6,14.0,9.7,NaN,NaN,NaN,None,None,None,NaN,NaN,6,f87bfae7-95b2-458e-9326-bf90287f701c,2025-03-02T20:08:05.157Z,2025-03-02T20:08:32.174Z,True,pitch,None,None,None,NaN,NaN,NaN,1
4,778935,2025-03-02,682998,Corbin Carroll,L,AZ,109,592351,Jon Gray,R,TEX,140,0,Swinging Strike,S,False,True,True,True,True,True,False,SL,Slider,2,2,0,3,2,0,88.1,81.2,3.117,1.573,100.08,198.92,4.219440,26.414158,-31.128767,2.535455,0.624380,0.443989,1.476112,4.219382,-128.186277,-4.207924,-1.552380,50.000087,5.568195,9,0.89,0.427243,6.200200,2443,137,-34.8,0.4,-5.5,NaN,NaN,NaN,None,None,None,NaN,NaN,7,35971e5e-9fc2-4d94-8e03-4d40000cb768,2025-03-02T20:08:34.915Z,2025-03-02T20:08:38.033Z,True,pitch,atBat,Strikeout,strikeout,0.0,0.0,0.0,1


In [9]:
df_pyb = df_pyb[(df_pyb["pitcher_name"] == "Luis Curvelo")]
df_pyb.shape

(15, 79)

In [39]:
pitcher_pyb = df_pyb[
    [
        "game_id",
        "game_date",
        "pitcher_name",
        "ab_number",
        "pitch_description",
        "balls",
        "strikes",
        "play_description",
        "PitchesThrown",
        "pfxz",
        "pfxx",
        "spin_rate",
        "start_speed",
        "ivb",
        "hb",
        "is_whiff",
        "play_description",
        "play_code",
        "zone",
        "is_swing",
        "is_strike",
        "ax",
        "ay",
        "az",
        "vx0",
        "vy0",
        "vz0",
        "extension",
        "launch_speed",
        "batter_name",
    ]
]

In [40]:
pitcher_pyb = pitcher_pyb.sort_values(by=["pitch_description"])
pitcher_pyb

,game_id,game_date,pitcher_name,ab_number,pitch_description,balls,strikes,play_description,PitchesThrown,pfxz,pfxx,spin_rate,start_speed,ivb,hb,is_whiff,play_description,play_code,zone,is_swing,is_strike,ax,ay,az,vx0,vy0,vz0,extension,launch_speed,batter_name
251,778935,2025-03-02,Drey Jameson,60,Changeup,0,0,Ball,1,1.458809,-10.341985,2056,91.0,1.8,16.9,None,Ball,B,13,None,False,-18.398921,27.161976,-29.584085,8.834704,-132.259523,-4.834475,5.499505,NaN,Jax Biggers
199,778935,2025-03-02,Bryce Jarvis,48,Changeup,3,2,Foul,1,1.603292,-10.295578,2065,84.1,2.8,17.4,None,Foul,F,7,True,True,-15.751425,22.449145,-29.720511,5.982345,-122.444404,-1.990885,6.010213,NaN,Jake Burger
36,778935,2025-03-02,Jon Gray,9,Changeup,0,1,Foul,1,5.636813,-8.419990,1398,87.3,9.7,13.9,None,Foul,F,4,True,True,-13.717603,26.934557,-22.989860,5.436925,-127.064274,-3.154054,6.158634,NaN,Trey Mancini
78,778935,2025-03-02,Blake Walston,20,Changeup,2,2,Ball,1,2.591470,6.897098,1244,84.4,3.6,-11.3,None,Ball,B,14,None,False,10.520199,24.602839,-28.215306,-5.196875,-122.806784,-5.666044,5.541630,NaN,Leody Taveras
31,778935,2025-03-02,Eduardo Rodriguez,7,Changeup,1,1,"In play, out(s)",1,2.487220,11.524033,1983,87.4,3.8,-19.7,None,"In play, out(s)",X,14,True,False,18.589197,29.438051,-28.157866,-4.239460,-127.098997,-3.483154,6.245625,52.8,Josh Jung
83,778935,2025-03-02,Blake Walston,22,Changeup,0,1,Ball,1,1.399934,9.896443,1264,85.3,1.3,-16.2,None,Ball,B,14,None,False,15.424177,24.294922,-29.983826,-7.576063,-123.931966,-6.136569,5.474049,NaN,Marcus Semien
209,778935,2025-03-02,Bryce Jarvis,50,Changeup,2,2,Swinging Strike (Blocked),1,2.541653,-11.269203,2082,85.2,3.6,17.9,True,Swinging Strike (Blocked),W,14,True,True,-17.395626,25.390580,-28.244418,10.227201,-123.656629,-4.809105,5.963879,NaN,Leody Taveras
185,778935,2025-03-02,Hoby Milner,46,Changeup,2,2,"In play, out(s)",1,0.432207,9.635564,1656,81.1,1.7,-14.5,None,"In play, out(s)",X,8,True,False,13.510182,21.483992,-31.571734,-12.594477,-117.437428,2.317333,6.661581,76.7,Trey Mancini
63,778935,2025-03-02,Eduardo Rodriguez,16,Changeup,1,1,"In play, out(s)",1,3.751426,7.464381,1980,87.7,6.0,-12.5,None,"In play, out(s)",X,8,True,False,12.288390,26.854085,-25.999495,-4.361800,-127.642723,-4.575087,6.169057,108.3,Kevin Pillar
20,778935,2025-03-02,Eduardo Rodriguez,5,Changeup,1,1,Ball,1,3.515641,8.519459,1933,85.8,5.3,-14.0,None,Ball,B,13,None,False,13.339651,26.672849,-26.669864,-5.700646,-124.759452,-5.372687,6.192180,NaN,Marcus Semien


In [41]:
is_ball = [11, 12, 13, 14]
strike = [1, 2, 3, 4, 5, 6, 7, 8, 9]
pitcher_pyb["InZone"] = np.where(pitcher_pyb["zone"].isin(strike), 1, 0)
pitcher_pyb["OutZone"] = np.where(pitcher_pyb["zone"].isin(is_ball), 1, 0)
pitcher_pyb

,game_id,game_date,pitcher_name,ab_number,pitch_description,balls,strikes,play_description,PitchesThrown,pfxz,pfxx,spin_rate,start_speed,ivb,hb,is_whiff,play_description,play_code,zone,is_swing,is_strike,ax,ay,az,vx0,vy0,vz0,extension,launch_speed,batter_name,InZone,OutZone
251,778935,2025-03-02,Drey Jameson,60,Changeup,0,0,Ball,1,1.458809,-10.341985,2056,91.0,1.8,16.9,None,Ball,B,13,None,False,-18.398921,27.161976,-29.584085,8.834704,-132.259523,-4.834475,5.499505,NaN,Jax Biggers,0,1
199,778935,2025-03-02,Bryce Jarvis,48,Changeup,3,2,Foul,1,1.603292,-10.295578,2065,84.1,2.8,17.4,None,Foul,F,7,True,True,-15.751425,22.449145,-29.720511,5.982345,-122.444404,-1.990885,6.010213,NaN,Jake Burger,1,0
36,778935,2025-03-02,Jon Gray,9,Changeup,0,1,Foul,1,5.636813,-8.419990,1398,87.3,9.7,13.9,None,Foul,F,4,True,True,-13.717603,26.934557,-22.989860,5.436925,-127.064274,-3.154054,6.158634,NaN,Trey Mancini,1,0
78,778935,2025-03-02,Blake Walston,20,Changeup,2,2,Ball,1,2.591470,6.897098,1244,84.4,3.6,-11.3,None,Ball,B,14,None,False,10.520199,24.602839,-28.215306,-5.196875,-122.806784,-5.666044,5.541630,NaN,Leody Taveras,0,1
31,778935,2025-03-02,Eduardo Rodriguez,7,Changeup,1,1,"In play, out(s)",1,2.487220,11.524033,1983,87.4,3.8,-19.7,None,"In play, out(s)",X,14,True,False,18.589197,29.438051,-28.157866,-4.239460,-127.098997,-3.483154,6.245625,52.8,Josh Jung,0,1
83,778935,2025-03-02,Blake Walston,22,Changeup,0,1,Ball,1,1.399934,9.896443,1264,85.3,1.3,-16.2,None,Ball,B,14,None,False,15.424177,24.294922,-29.983826,-7.576063,-123.931966,-6.136569,5.474049,NaN,Marcus Semien,0,1
209,778935,2025-03-02,Bryce Jarvis,50,Changeup,2,2,Swinging Strike (Blocked),1,2.541653,-11.269203,2082,85.2,3.6,17.9,True,Swinging Strike (Blocked),W,14,True,True,-17.395626,25.390580,-28.244418,10.227201,-123.656629,-4.809105,5.963879,NaN,Leody Taveras,0,1
185,778935,2025-03-02,Hoby Milner,46,Changeup,2,2,"In play, out(s)",1,0.432207,9.635564,1656,81.1,1.7,-14.5,None,"In play, out(s)",X,8,True,False,13.510182,21.483992,-31.571734,-12.594477,-117.437428,2.317333,6.661581,76.7,Trey Mancini,1,0
63,778935,2025-03-02,Eduardo Rodriguez,16,Changeup,1,1,"In play, out(s)",1,3.751426,7.464381,1980,87.7,6.0,-12.5,None,"In play, out(s)",X,8,True,False,12.288390,26.854085,-25.999495,-4.361800,-127.642723,-4.575087,6.169057,108.3,Kevin Pillar,1,0
20,778935,2025-03-02,Eduardo Rodriguez,5,Changeup,1,1,Ball,1,3.515641,8.519459,1933,85.8,5.3,-14.0,None,Ball,B,13,None,False,13.339651,26.672849,-26.669864,-5.700646,-124.759452,-5.372687,6.192180,NaN,Marcus Semien,0,1


In [42]:
pitcher_pyb["vy_f"] = -np.sqrt(
    pitcher_pyb["vy0"] ** 2 - (2 * pitcher_pyb["ay"] * (y0 - yf))
)

# Compute time (t)
pitcher_pyb["t"] = (pitcher_pyb["vy_f"] - pitcher_pyb["vy0"]) / pitcher_pyb["ay"]

# Compute final z-velocity (vz_f)
pitcher_pyb["vz_f"] = pitcher_pyb["vz0"] + (pitcher_pyb["az"] * pitcher_pyb["t"])

# Compute final x-velocity (vx_f)
pitcher_pyb["vx_f"] = pitcher_pyb["vx0"] + (pitcher_pyb["ax"] * pitcher_pyb["t"])

# Compute VAA
pitcher_pyb["VAA"] = -np.arctan(pitcher_pyb["vz_f"] / pitcher_pyb["vy_f"]) * (
    180 / np.pi
)

# Compute Horizontal Approach Angle (HAA)
pitcher_pyb["HAA"] = -np.arctan(pitcher_pyb["vx_f"] / pitcher_pyb["vy_f"]) * (
    180 / np.pi
)

# Get average vRel per pitch type
pitch_type_vrel = (
    df_pyb.groupby("pitch_description", as_index=False)["z0"].mean()
).round(1)
pitch_type_vrel.rename(columns={"z0": "vRel"}, inplace=True)

# Get average hRel per pitch type
pitch_type_hrel = (
    df_pyb.groupby("pitch_description", as_index=False)["x0"].mean()
).round(1)
pitch_type_hrel.rename(columns={"x0": "hRel"}, inplace=True)
pitcher_hand_unique = df_pyb[["pitch_description", "pitcher_hand"]].drop_duplicates(
    subset=["pitch_description"]
)
pitch_type_hrel = pitch_type_hrel.merge(
    pitcher_hand_unique, on="pitch_description", how="left"
)
pitch_type_hrel["hRel"] = np.where(
    pitch_type_hrel["pitcher_hand"] == "L",
    -pitch_type_hrel["hRel"],
    pitch_type_hrel["hRel"],
)

whiff_pitches = pitcher_pyb[pitcher_pyb["is_whiff"] == True]
pitch_type_whiff = (
    whiff_pitches.groupby("pitch_description").size().reset_index(name="whiff_count")
)
pitch_type_whiff.rename(columns={"whiff_count": "Whiffs"}, inplace=True)
pitch_type_whiff["Whiffs"] = pitch_type_whiff["Whiffs"].astype(int)

strike_pitches = pitcher_pyb[pitcher_pyb["play_code"] == "C"]
pitch_type_cs = (
    strike_pitches.groupby("pitch_description").size().reset_index(name="CS")
)

pitches_in_zone = pitcher_pyb.groupby("pitch_description")["InZone"].sum().reset_index()
pitches_in_zone.rename(columns={"InZone": "Pitches_In_Zone"}, inplace=True)

pitches_out_of_zone = (
    pitcher_pyb.groupby("pitch_description")["OutZone"].sum().reset_index()
)
pitches_out_of_zone.rename(columns={"OutZone": "Pitches_Out_Of_Zone"}, inplace=True)
# print(pitches_out_of_zone)


swings_out_of_zone = pitcher_pyb[
    pitcher_pyb["zone"].isin([11, 12, 13, 14]) & pitcher_pyb["is_swing"] == True
]
swings_out_of_zone = (
    swings_out_of_zone.groupby("pitch_description")["is_swing"]
    .sum()
    .astype(int)  # Convert to integer
    .reset_index()
)
# print(swings_out_of_zone)


pitch_type_spin = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["spin_rate"].mean()
).round(1)
pitch_type_spin.rename(columns={"spin_rate": "Spin Rate"}, inplace=True)

pitch_type_swing = (
    pitcher_pyb.groupby("pitch_description")["is_swing"].sum().astype(int).reset_index()
)
pitch_type_swing.rename(columns={"is_swing": "Swings"}, inplace=True)


pitch_type_ivb = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["ivb"].mean()
).round(1)
pitch_type_ivb.rename(columns={"ivb": "iVB"}, inplace=True)

pitch_type_hb = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["hb"].mean()
).round(1)
pitch_type_hb.rename(columns={"hb": "HB"}, inplace=True)

pitch_avg_velo = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["start_speed"].mean()
).round(1)
pitch_avg_velo.rename(columns={"start_speed": "Avg Velo"}, inplace=True)

pitch_avg_exten = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["extension"].mean()
).round(1)
pitch_avg_exten.rename(columns={"extension": "Extension"}, inplace=True)

# Compute the mean VAA for each pitch type
vaa_means = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["VAA"].mean()
).round(1)

# Compute the mean HAA for each pitch type
haa_means = (
    pitcher_pyb.groupby("pitch_description", as_index=False)["HAA"].mean()
).round(1)

# Compute the highest exit velocity for each pitch type
df_hits = pitcher_pyb.dropna(subset=["launch_speed"])
# Group by pitch type and find the index of the max exit velocity, handling NaN values
idx = df_hits.groupby("pitch_description")["launch_speed"].idxmax().dropna()
# Retrieve the rows with max exit velocity
max_exit_velo = df_hits.loc[
    idx, ["pitch_description", "batter_name", "launch_speed"]
].copy()
max_exit_velo.rename(columns={"launch_speed": "Max Exit Velo"}, inplace=True)

pitch_type_counts = pitcher_pyb.groupby(
    ["pitcher_name", "pitch_description"], as_index=False
)["PitchesThrown"].sum()

In [43]:
pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown
0,Blake Walston,Changeup,4
1,Blake Walston,Curveball,6
2,Blake Walston,Cutter,5
3,Blake Walston,Four-Seam Fastball,14
4,Blake Walston,Sinker,2
5,Blake Walston,Sweeper,11
6,Bryce Jarvis,Changeup,5
7,Bryce Jarvis,Cutter,1
8,Bryce Jarvis,Four-Seam Fastball,13
9,Bryce Jarvis,Sinker,3


In [37]:
pitch_type_counts["Total Pitches"] = pitch_type_counts["PitchesThrown"].sum()
pitch_type_counts

,pitcher_name,pitch_description,PitchesThrown,Total Pitches
0,Luis Curvelo,Four-Seam Fastball,7,15
1,Luis Curvelo,Slider,8,15


In [38]:
pitch_type_counts["Usage"] = (
    (pitch_type_counts["PitchesThrown"] / pitch_type_counts["Total Pitches"]) * 100
).round(2)

pitch_type_counts = (
    pitch_type_counts.merge(pitch_type_spin, on="pitch_description", how="left")
    .merge(pitch_avg_velo, on="pitch_description", how="left")
    .merge(pitch_type_ivb, on="pitch_description", how="left")
    .merge(pitch_type_hb, on="pitch_description", how="left")
    .merge(pitch_type_whiff, on="pitch_description", how="left")
    .merge(pitch_type_cs, on="pitch_description", how="left")
    .merge(pitches_in_zone, on="pitch_description", how="left")
    .merge(pitches_out_of_zone, on="pitch_description", how="left")
    .merge(swings_out_of_zone, on="pitch_description", how="left")
    .merge(pitch_type_swing, on="pitch_description", how="left")
    .merge(pitch_type_vrel, on="pitch_description", how="left")
    .merge(pitch_type_hrel, on="pitch_description", how="left")
    .merge(vaa_means, on="pitch_description", how="left")
    .merge(haa_means, on="pitch_description", how="left")
    .merge(pitch_avg_exten, on="pitch_description", how="left")
    .merge(max_exit_velo, on="pitch_description", how="left")
)

pitch_type_counts = pitch_type_counts.sort_values(by="PitchesThrown", ascending=False)

pitch_type_counts["Whiffs"] = pitch_type_counts["Whiffs"].fillna(0).astype(int)
pitch_type_counts["CS"] = pitch_type_counts["CS"].fillna(0).astype(int)
pitch_type_counts["CS+Whiffs"] = pitch_type_counts["CS"] + pitch_type_counts["Whiffs"]
pitch_type_counts["Zone%"] = (
    (pitch_type_counts["Pitches_In_Zone"] / pitch_type_counts["PitchesThrown"]) * 100
).round(1)
pitch_type_counts["is_swing"] = pitch_type_counts["is_swing"].fillna(0).astype(int)
pitch_type_counts["Chase%"] = (
    (pitch_type_counts["is_swing"] / pitch_type_counts["Pitches_Out_Of_Zone"]) * 100
).round(1)
pitch_type_counts["Whiff%"] = (
    (pitch_type_counts["Whiffs"] / pitch_type_counts["Swings"]) * 100
).round(1)


pitch_type_counts = pitch_type_counts[
    [
        "pitcher_name",
        "pitch_description",
        "PitchesThrown",
        "Usage",
        "Spin Rate",
        "Avg Velo",
        "iVB",
        "HB",
        "Whiffs",
        "CS",
        "CS+Whiffs",
        "Zone%",
        "Chase%",
        "Whiff%",
        "vRel",
        "hRel",
        "VAA",
        "HAA",
        "Extension",
        "game_date",
    ]
]


pitch_type_counts.rename(
    columns={
        "game_date": "Date",
        "PitchesThrown": "Count",
        "pitch_description": "Pitch Type",
        "batter_name": "Batter",
        "pitcher_name": "Pitcher",
    },
    inplace=True,
)


print(pitch_type_counts)


KeyError: "['game_date'] not in index"